# How it will Work?
#### For Server
##### Enter Username: mm_team04_02 
##### Enter Password: mm_team04_02Pass-

#### For localhost
##### Enter Username: root
##### Click on Login 
##### It will pop a window to Enter MySQL password, so if there is any localhost password please enter that and click on OK
##### And if there is no password just click on OK on that window. 


##### Now the first window will show which Query you need to check 1 to 6. Please enter any value between 1 to 6 and Select Run Query.

##### The pop up window will ask either you want full data to be displayed or you want any specific detail. If you slected YES you will get full data.

##### If you will NO, next Pop Up window will open and it will ask Specific Data point to be inserted and the details of that will be displayed.

##### At last please click on close so that it terminates the application. 

##### If you will enter any wrong number while SELECTING query like 7 it will throw an error and you have enter value again.

##### If you enter any wrong detail on the specific data entry. it will also throw an error and you have to run the Query again.

# Important Points:
##### Range of Store_IDs : 1211 - 1230
##### Range of ZIP Code: 98001 - 98020 
##### Range of Product_ID: 10001 - 10201
##### Range of Supplier_ID: 1111 - 1161
##### Product Categories: ELECTRONICS, BEVERAGES, HYGIENE, BOOKS & MEDIA, KIDS

In [9]:
import tkinter as tk
from tkinter import ttk, simpledialog, messagebox
import mysql.connector

# Global variables for database connection
conn = None
cursor = None

def login():
    global conn, cursor

    # Getting username
    username = username_entry.get()

    # Connecting to MySQL database
    try:
        if username.lower() == 'root':
            password = simpledialog.askstring("Password", "Enter MySQL password:")
            if password is None:
                return  # Return if the user cancels password entry
            conn = mysql.connector.connect(
                host="localhost",
                user=username,
                password=password,
                database="mm_team04_02"
            )
        else:
            password = password_entry.get()
            conn = mysql.connector.connect(
                host="CSSQL",
                user=username,
                password=password,
                database="mm_team04_02"
            )
        cursor = conn.cursor()

        # Closing the login window
        login_window.destroy()

        # Showing the main window
        root.deiconify()

    except mysql.connector.Error as e:
        messagebox.showerror("Error", f"Unable to connect to the database: {e}")
    except Exception as e:
        messagebox.showerror("Error", f"An unexpected error occurred: {e}")

# Function to execute selected SQL query
def execute_selected_query():
    selected_query = int(query_entry.get())
    if 1 <= selected_query <= 6:
        try:
            if tk.messagebox.askyesno("Data Option", "Do you want to retrieve the full data?"):
                if selected_query == 1:
                    query = "SELECT s.SUPPLIER_ID, s.SUPPLIER_NAME, s.ADDRESS AS SUPPLIER_ADDRESS, p.PRODUCT_NAME, p.PRODUCT_ID \
                             FROM supplier s \
                             JOIN product p ON s.SUPPLIER_ID = p.SUPPLIER_ID;"
                elif selected_query == 2:
                    query = "SELECT st.STORE_ID, st.ZIP_CODE, st.STATE, SUM(s.TOTAL_AMOUNT) AS TOTAL_SALES_AMOUNT, \
                             SUM(p.GROSS_PROFIT) AS TOTAL_GROSS_PROFIT, SUM(p.TOTAL_EXPENSES) AS TOTAL_EXPENSES, \
                             SUM(p.NET_PROFIT) AS TOTAL_NET_PROFIT \
                             FROM sales s \
                             JOIN store st ON s.STORE_ID = st.STORE_ID \
                             JOIN profitability p ON p.STORE_ID = st.STORE_ID \
                             GROUP BY st.STORE_ID, st.ZIP_CODE, st.STATE;"
                elif selected_query == 3:
                    query = "SELECT p.CATEGORY, \
                             SUM(w.WAREHOUSE_QUANTITY) AS TOTAL_QUANTITY, \
                             SUM(w.WAREHOUSE_QUANTITY * p.COST_PRICE) AS TOTAL_COST_OF_MATERIAL_IN_WAREHOUSE \
                             FROM product p \
                             JOIN warehouse w ON p.PRODUCT_ID = w.PRODUCT_ID \
                             GROUP BY p.CATEGORY \
                             ORDER BY TOTAL_QUANTITY DESC;"
                elif selected_query == 4:
                    query = "SELECT e.STORE_ID, e.EMPLOYEE_ID, e.EMP_NAME, e.POSITION, e.EMAIL, e.PHONE_NUMBER, e.HIRE_DATE \
                             FROM employee e \
                             JOIN ( \
                                 SELECT STORE_ID, MIN(HIRE_DATE) AS SENIOR_DATE \
                                 FROM employee \
                                 GROUP BY STORE_ID \
                             ) AS t ON e.STORE_ID = t.STORE_ID AND e.HIRE_DATE = t.SENIOR_DATE;"
                elif selected_query == 5:
                    query = "SELECT p.PRODUCT_ID, p.PRODUCT_NAME, p.CATEGORY, \
                             COUNT(f.PRODUCT_ID) AS FEEDBACK_COUNT, \
                             AVG(f.RATING) AS AVERAGE_RATING, \
                             SUM(p.QUANTITY) AS TOTAL_QUANTITY_AVAILABLE \
                             FROM product p \
                             LEFT JOIN feedback f ON p.PRODUCT_ID = f.PRODUCT_ID \
                             GROUP BY p.PRODUCT_ID, p.PRODUCT_NAME, p.CATEGORY \
                             ORDER BY FEEDBACK_COUNT DESC;"
                elif selected_query == 6:
                    query = "SELECT st.STORE_ID, st.ZIP_CODE, st.STATE, DATE_FORMAT(s.SALES_DATE, '%m-%Y') AS Month, \
                             COUNT(*) AS TotalSalesCount, \
                             AVG(s.TOTAL_AMOUNT) AS AvgTotalSalesAmount \
                             FROM sales s \
                             JOIN store st ON s.STORE_ID = st.STORE_ID \
                             GROUP BY st.STORE_ID, st.ZIP_CODE, st.STATE, Month \
                             ORDER BY st.STORE_ID, Month;"
                
                cursor.execute(query)
                columns = [desc[0] for desc in cursor.description]
                data = cursor.fetchall()

                if data:
                    # Clearing previous results
                    for row in result_tree.get_children():
                        result_tree.delete(row)
                    result_tree.column("#0",width=30)

                    # Inserting new data
                    for row in data:
                        result_tree.insert('', 'end', values=row)

                    # Updating column headings
                    result_tree['columns'] = columns
                    for col in columns:
                        result_tree.heading(col, text=col)

                    result_label.config(text="Query executed successfully")
                else:
                    messagebox.showerror("Error", "No data found for the given criteria.")
            else:
                input_data = ""
                if selected_query == 1:
                    input_data = tk.simpledialog.askstring("Input", "Enter Supplier ID / Supplier Name / Product ID / Product Name:")
                elif selected_query == 2:
                    input_data = tk.simpledialog.askstring("Input", "Enter Store ID / ZIP Code:")
                elif selected_query == 3:
                    input_data = tk.simpledialog.askstring("Input", "Enter Product ID / Category:")
                elif selected_query == 4:
                    input_data = tk.simpledialog.askstring("Input", "Enter Store ID / Employee ID:")
                elif selected_query == 5:
                    input_data = tk.simpledialog.askstring("Input", "Enter Product ID / Product Name / Category:")
                elif selected_query == 6:
                    input_data = tk.simpledialog.askstring("Input", "Enter Store ID / ZIP Code:")
                
                if input_data:
                    query = ""
                    if selected_query == 1:
                        query = f"SELECT s.SUPPLIER_ID, p.PRODUCT_ID, s.SUPPLIER_NAME, p.PRODUCT_NAME, s.ADDRESS AS SUPPLIER_ADDRESS  \
                                  FROM supplier s \
                                  JOIN product p ON s.SUPPLIER_ID = p.SUPPLIER_ID \
                                  WHERE s.SUPPLIER_ID = '{input_data}' \
                                  OR s.SUPPLIER_NAME = '{input_data}' \
                                  OR p.PRODUCT_ID = '{input_data}' \
                                  OR p.PRODUCT_NAME = '{input_data}';"
                    elif selected_query == 2:
                        query = f"SELECT st.STORE_ID, st.ZIP_CODE, st.STATE, \
                                  SUM(s.TOTAL_AMOUNT) AS TOTAL_SALES_AMOUNT, \
                                  SUM(p.GROSS_PROFIT) AS TOTAL_GROSS_PROFIT, \
                                  SUM(p.TOTAL_EXPENSES) AS TOTAL_EXPENSES, \
                                  SUM(p.NET_PROFIT) AS TOTAL_NET_PROFIT \
                                  FROM sales s \
                                  JOIN store st ON s.STORE_ID = st.STORE_ID \
                                  JOIN profitability p ON p.STORE_ID = st.STORE_ID \
                                  WHERE st.STORE_ID = '{input_data}' \
                                  OR st.ZIP_CODE = '{input_data}' \
                                  GROUP BY st.STORE_ID, st.ZIP_CODE, st.STATE;"
                    elif selected_query == 3:
                        query = f"SELECT p.CATEGORY, \
                                  SUM(w.WAREHOUSE_QUANTITY) AS TOTAL_QUANTITY, \
                                  SUM(w.WAREHOUSE_QUANTITY * p.COST_PRICE) AS TOTAL_COST_OF_MATERIAL_IN_WAREHOUSE \
                                  FROM product p \
                                  JOIN warehouse w ON p.PRODUCT_ID = w.PRODUCT_ID \
                                  WHERE p.PRODUCT_ID = '{input_data}' \
                                  OR p.CATEGORY = '{input_data}' \
                                  GROUP BY p.CATEGORY \
                                  ORDER BY TOTAL_QUANTITY DESC;"
                    elif selected_query == 4:
                        query = f"SELECT e.STORE_ID, e.EMPLOYEE_ID, e.EMP_NAME, e.POSITION, e.EMAIL, e.PHONE_NUMBER, e.HIRE_DATE \
                                  FROM employee e \
                                  JOIN ( \
                                      SELECT STORE_ID, MIN(HIRE_DATE) AS SENIOR_DATE \
                                      FROM employee \
                                      GROUP BY STORE_ID \
                                  ) AS t ON e.STORE_ID = t.STORE_ID AND e.HIRE_DATE = t.SENIOR_DATE \
                                  WHERE e.STORE_ID = '{input_data}' OR e.EMPLOYEE_ID = '{input_data}';"
                    elif selected_query == 5:
                        query = f"SELECT p.PRODUCT_ID, p.PRODUCT_NAME, p.CATEGORY, \
                                  COUNT(f.PRODUCT_ID) AS FEEDBACK_COUNT, \
                                  AVG(f.RATING) AS AVERAGE_RATING, \
                                  SUM(p.QUANTITY) AS TOTAL_QUANTITY_AVAILABLE \
                                  FROM product p \
                                  LEFT JOIN feedback f ON p.PRODUCT_ID = f.PRODUCT_ID \
                                  WHERE p.PRODUCT_ID = '{input_data}' \
                                  OR p.PRODUCT_NAME = '{input_data}' \
                                  OR p.CATEGORY = '{input_data}' \
                                  GROUP BY p.PRODUCT_ID, p.PRODUCT_NAME, p.CATEGORY \
                                  ORDER BY FEEDBACK_COUNT DESC;"
                    elif selected_query == 6:
                        query = f"SELECT st.STORE_ID, st.ZIP_CODE, st.STATE, DATE_FORMAT(s.SALES_DATE, '%m-%Y') AS Month, \
                                  COUNT(*) AS TotalSalesCount, \
                                  AVG(s.TOTAL_AMOUNT) AS AvgTotalSalesAmount \
                                  FROM sales s \
                                  JOIN store st ON s.STORE_ID = st.STORE_ID \
                                  WHERE st.STORE_ID = '{input_data}' OR st.ZIP_CODE = '{input_data}' \
                                  GROUP BY st.STORE_ID, st.ZIP_CODE, st.STATE, Month \
                                  ORDER BY st.STORE_ID, Month;"
                    
                    if query:
                        cursor.execute(query)
                        columns = [desc[0] for desc in cursor.description]
                        data = cursor.fetchall()

                        if data:
                            # Clearing previous results
                            for row in result_tree.get_children():
                                result_tree.delete(row)
                            result_tree.column("#0",width=30)

                            # Inserting new data
                            for row in data:
                                result_tree.insert('', 'end', values=row)

                            # Updating column headings
                            result_tree['columns'] = columns
                            for col in columns:
                                result_tree.heading(col, text=col)

                            result_label.config(text="Query executed successfully")
                        else:
                            messagebox.showerror("Error", "No data found for the given criteria.")
                else:
                    messagebox.showerror("Error", "Invalid input data.")
                    
        except mysql.connector.Error as e:
            messagebox.showerror("Error", f"MySQL Error: {e}")
        except Exception as e:
            result_label.config(text=f"Error: {e}")
        

    else:
        tk.messagebox.showerror("Error", "Invalid query number. Please enter a number between 1 and 6.")

# Creating main window
root = tk.Tk()
root.title("MySQL Query Tool")

# Creating login window
login_window = tk.Toplevel(root)
login_window.title("Login")

# Creating labels and entry widgets for username and password
tk.Label(login_window, text="Username:").grid(row=0, column=0, padx=10, pady=5)
username_entry = tk.Entry(login_window)
username_entry.grid(row=0, column=1, padx=10, pady=5)

tk.Label(login_window, text="Password:").grid(row=1, column=0, padx=10, pady=5)
password_entry = tk.Entry(login_window, show="*")
password_entry.grid(row=1, column=1, padx=10, pady=5)

# Creating login button
login_button = ttk.Button(login_window, text="Login", command=login)
login_button.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

# Hiding the main window until login is successful
root.withdraw()

# Creating main window components
# Label for query selection
query_label = ttk.Label(root, text="Enter the query number you wish to check (1 to 6):")
query_label.grid(row=0, column=0, padx=10, pady=10, sticky="w")

# Entries for query number
query_entry = ttk.Entry(root)
query_entry.grid(row=0, column=1, padx=10, pady=10, sticky="w")

# Labeling for query descriptions
query_descriptions = [
    "Retrieve supplier details along with their products.",
    "Retrieve store details along with sales and profitability information.",
    "Retrieve total quantity and cost of materials in the warehouse for each category of products.",
    "Retrieve details of the most senior employee in each store.",
    "Retrieve feedback count, average rating, and total quantity available for each product.",
    "Retrieve total sales count and average total sales amount for each store in a given month."
]

for i, description in enumerate(query_descriptions, start=1):
    description_label = ttk.Label(root, text=f"Query {i}: {description}")
    description_label.grid(row=i, column=0, columnspan=2, padx=10, pady=5, sticky="w")

# Button to run query
query_button = ttk.Button(root, text="Run Query", command=execute_selected_query)
query_button.grid(row=len(query_descriptions) + 1, column=0, columnspan=2, padx=10, pady=10, sticky="w")

# Treeview to display results
result_tree = ttk.Treeview(root)
result_tree.grid(row=len(query_descriptions) + 2, column=0, columnspan=2, padx=10, pady=10, sticky="w")

# Labelling for error messages
result_label = ttk.Label(root, text="")
result_label.grid(row=len(query_descriptions) + 3, column=0, columnspan=2, padx=10, pady=5, sticky="w")

def close_window():
    root.destroy()
    root.quit()   

# Button to close the window
close_button = ttk.Button(root, text="Close", command=close_window)
close_button.grid(row=len(query_descriptions) + 4, column=0, columnspan=2, padx=10, pady=10, sticky="w")

# Run the application
root.mainloop()

